## Data Pre-processing


Data pre-processing techniques refer to addition, deletion, or transformation of training set data. Data prepration can make or break a model's preditive ability. 

The need for data pre-processing is determined by the type of model being used. Some models are less or more sensitive to data. 

**Unsupervised** data models (e.g. PCA) do not consider the outcome variable in the pre-processing, wherease, the **supervised** models take them into consideration (e.g. partial least squares).

Feature enginerring - how the predictors are encoded, can have a significant impact on model performance. For example, using combination of predictors can sometimes be more effective than using the individual values: the ration of two predictors may be more effective than using two independent predictors. 


### Case Study: Cell Segmentation in High-Content Screening

To measure the effects of medicines or diseases on the size, shape, development status, and number of cells in living organizm or plant, experts, examines the target serum or tissue under miscroscope and maually assess the desired characteristis. This processes is tedious and a better approach is to use high-contrast screening. 

The high-contrast screening works by measuring the light reflected from a stain that binds to specific characteristic of cell such as cell nuclei. However, this automated method often results in misleading measurements. 

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

%matplotlib inline  


In [5]:
datasets = glob.glob('./datasets/FuelEconomy/*.csv')

In [6]:
datasets

['./datasets/FuelEconomy/cars2011.csv',
 './datasets/FuelEconomy/cars2010.csv',
 './datasets/FuelEconomy/cars2012.csv']